In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import pudl
import pudl.constants as pc
import sqlalchemy as sa
import logging
import pathlib
from copy import deepcopy
import sys

from fuzzywuzzy import fuzz, process
from rmi_pudl.deprish import *

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

pd.options.display.max_columns = None

In [ ]:
pudl_settings = pudl.workspace.setup.get_defaults()
ferc1_engine = sa.create_engine(pudl_settings["ferc1_db"])
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])

In [ ]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine, freq='AS', fill_net_gen=False)

### Process Depreciation Data

In [ ]:
file_path_deprish = pathlib.Path().cwd().parent/'inputs'/'depreciation_rmi.xlsx'

# get the studies from the spreadsheet
extract_df = Extractor(
    file_path=file_path_deprish,
    sheet_name=0).execute()

# create a transformer object to process the extracted data
# 1. tidying w/ early_tidy()
# 2. reshaping w/ reshape() - this is where the 
# 3. filling-in w/ fill_in()
transformer = Transformer(
    extract_df=extract_df
)

In [ ]:
# the transformer has 3 main steps that are encapsulated in execute
transform_df = transformer.execute()

In [ ]:
transform_df.head()

### Read and process Common Labels

In [ ]:
file_path_deprish = pathlib.Path().cwd().parent/'inputs'/'depreciation_rmi.xlsx'

In [ ]:
common_assn = make_default_common_assn(file_path_deprish)

In [ ]:
common_assn_wide = transform_common_assn_for_labeling(common_assn)

In [ ]:
common_labeling = make_common_assn_for_labeling(common_assn, pudl_out, transformer)

In [ ]:
path_common_assn=  pathlib.Path().cwd().parent/'outputs' /'common_assn.csv'
common_assn.to_csv(path_common_assn,index=False)

In [ ]:
path_common_labeling=  pathlib.Path().cwd().parent/'outputs' /'common_labeling.csv'
common_labeling.to_csv(path_common_labeling)